In [1]:
import pandas as pd 
import torch 
import monai
import os
import pickle

In [2]:
path = "/data/kaplinsp/models/log_DenseNet121_2025-03-24_19.38.03.log-middle-slice-2d-0/"
model = monai.networks.nets.densenet121(
                dropout_prob=0.4, spatial_dims=3, in_channels=1, out_channels=2
            )
model = model.to(0)

In [3]:
model.load_state_dict(torch.load(os.path.join(path, "best_loss_model.pth"))["model_state_dict"])

/tmp/ipykernel_4105114/2537844734.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(os.path.join(path, "best_loss_model.pth"))["model_stat

<All keys matched successfully>

In [3]:
from datasets import getNoduleInfoList
with open(os.path.join(path, "val.pkl"), 'rb') as f:
    val_ids = pickle.load(f)
nodules = getNoduleInfoList(['sclc'])
nodules = [nod for nod in nodules if nod.is_nodule == 2]

In [5]:
from datasets import NoduleDataset
dataset = NoduleDataset(nodules, 
            isValSet_bool=True,
            dilate=20,
            resample=[128, 128, 128],
            box_size=40)

from torch.utils.data import DataLoader
val_dl = DataLoader(
            dataset,
            batch_size=16,
            num_workers=4,
            pin_memory=True,
            drop_last=False,
            shuffle=True,
        )

2025-04-03 20:33:30,578 INFO     pid:4105114 datasets:182:__init__ <datasets.NoduleDataset object at 0x7f9293a0a140>: 18 validation samples


In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Assuming you already have a model and a dataloader
model.eval()
all_probs = []
all_labels = []
all_paths = []

with torch.no_grad():
    for inputs, labels in val_dl:
        inputs = inputs.to(0)
        labels = torch.clip(labels.to(0), min=0, max=1)
        all_paths.extend([])

        outputs = model(inputs)  # Shape: (batch_size, num_classes) or (batch_size,)
        probs = torch.softmax(outputs, dim=1)[:, 1]  # For binary classification
        # If you're using nn.Sigmoid as the final layer, you don't need to apply it again

        all_probs.append(probs.cpu())
        all_labels.append(labels.cpu())

# Concatenate all batches
all_probs = torch.cat(all_probs).numpy()
all_labels = torch.cat(all_labels).numpy()

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(all_labels, all_probs)
roc_auc = auc(fpr, tpr)

/data/kaplinsp/envs/lminfer/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1174: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(


In [29]:
cutoff = 0.3570  # replace with your actual cutoff
pred_labels = (all_probs >= cutoff).astype(int)
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(all_labels, pred_labels))
print(classification_report(all_labels, pred_labels))

[[ 0  0]
 [10  8]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.44      0.62        18

    accuracy                           0.44        18
   macro avg       0.50      0.22      0.31        18
weighted avg       1.00      0.44      0.62        18



/data/kaplinsp/envs/lminfer/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/data/kaplinsp/envs/lminfer/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/data/kaplinsp/envs/lminfer/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [8]:
from datasets import getCtRawNodule
from image import NoSeg
has_seg = []
no_seg = []
for nod in nodules:
    try:
        getCtRawNodule(
            nod.file_path,
            nod.image_type,
            nod.center_lps, 
            True,
            20,
            [128, 128, 128],
            40
        )
        has_seg.append((nod.file_path, nod.center_lps))
    except NoSeg:
        no_seg.append((nod.file_path, nod.center_lps))
        

2025-04-03 20:51:21,146 INFO     pid:4118020 datasets:110:getCtRawNodule Slicing nodule from image for /data/kaplinsp/transformation/T462655SB.nrrd
2025-04-03 20:51:21,233 INFO     pid:4118020 datasets:110:getCtRawNodule Slicing nodule from image for /data/kaplinsp/transformation/T462657AB.nrrd
2025-04-03 20:51:21,312 INFO     pid:4118020 datasets:110:getCtRawNodule Slicing nodule from image for /data/kaplinsp/transformation/T462657AB.nrrd
2025-04-03 20:51:21,396 INFO     pid:4118020 datasets:110:getCtRawNodule Slicing nodule from image for /data/kaplinsp/transformation/T462658AB.nrrd
2025-04-03 20:51:21,488 INFO     pid:4118020 datasets:110:getCtRawNodule Slicing nodule from image for /data/kaplinsp/transformation/T462659SB.nrrd


In [10]:
no_seg

[('/data/kaplinsp/transformation/T462655SB.nrrd',
  (96.0378631066225, 10.633173432450292, -121.875)),
 ('/data/kaplinsp/transformation/T462657AB.nrrd',
  (-66.14879966887416, 21.47636589403976, -149.37499999999994)),
 ('/data/kaplinsp/transformation/T462657AB.nrrd',
  (-71.10774006622515, 28.342591059602626, -140.62499999999994)),
 ('/data/kaplinsp/transformation/T462658AB.nrrd',
  (-56.317673688741706, 80.91284244370866, -117.62499999999996)),
 ('/data/kaplinsp/transformation/T462659SB.nrrd',
  (-54.5680700112583, -66.75151193973505, -200.625))]